In [ ]:
import pandas as pd # type: ignore
import numpy as np # type: ignore
from keras.models import Sequential # type: ignore
from keras.layers import LSTM, Dense # type: ignore
from sklearn.preprocessing import MinMaxScaler # type: ignore
import pyodbc # type: ignore
from sklearn.model_selection import train_test_split # type: ignore

connection = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                            "Server=MIRACLE\\MIRACOZTURK;"
                            "Database=StokTahmin;"
                            "uid=X;pwd=Y")
df = pd.read_sql_query('SELECT Tarih, StokKodu, SUM(Miktar) AS Miktar FROM StokTahmin.dbo.depo_stok GROUP BY Tarih, StokKodu ORDER BY Tarih DESC', connection)

df['Tarih'] = pd.to_datetime(df['Tarih'])
df.sort_values(['StokKodu', 'Tarih'], inplace=True)

scaler = MinMaxScaler()

results = {}
look_back = 60

for stock_code in df['StokKodu'].unique():
    data = df[df['StokKodu'] == stock_code]
    if len(data) > 90:
        data['Miktar_scaled'] = scaler.fit_transform(data[['Miktar']].values.reshape(-1, 1))
        
        features = []
        labels = []

        for i in range(look_back, len(data)-1):
            features.append(data['Miktar_scaled'].values[i-look_back:i])
            labels.append(data['Miktar_scaled'].values[i])

        features = np.array(features)
        labels = np.array(labels)

        X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, shuffle=False)

        model = Sequential()
        model.add(LSTM(50, input_shape=(look_back, 1)))
        model.add(Dense(1))
        model.compile(optimizer='rmsprop', loss='mean_squared_error')

        X_train = X_train.reshape(X_train.shape[0], look_back, 1)
        model.fit(X_train, y_train, epochs=50, batch_size=1, verbose=0)

        last_sequence = data['Miktar_scaled'].values[-look_back:]
        last_sequence = last_sequence.reshape(1, look_back, 1)
        future_stock = model.predict(last_sequence)
        future_stock = scaler.inverse_transform(future_stock)

        last_date = data['Tarih'].iloc[-1]
        last_stock = data['Miktar'].iloc[-1]

        if last_stock == 0:
            results[stock_code] = "Stok zaten tukenmis."
        else:
            rate_of_depletion = future_stock[0, 0] / last_stock
            if np.isinf(rate_of_depletion):
                results[stock_code] = "Tahmin edilemiyor."
            else:
                days_to_deplete = int(np.ceil(rate_of_depletion))
                depletion_date = last_date + pd.Timedelta(days=days_to_deplete)
                results[stock_code] = depletion_date.strftime('%Y-%m-%d')

for stock_code, depletion_date in results.items():
    print(f"Stok Kodu {stock_code} icin stoklarin bitecegi tahmini tarih: {depletion_date}")